This notebook provides code to test all models with validation datasets (either held out test sets or external validation datasets).

In [1]:
# import statements 
import sys
sys.path.insert(1, './BioAutoMATED/main_classes/')
sys.path.append('./BioAutoMATED')
from wrapper import run_bioautomated
from integrated_design_helpers import *
from generic_automl_classes import convert_generic_input, read_in_data_file
from generic_deepswarm import print_summary
from transfer_learning_helpers import transform_classification_target, transform_regression_target, fit_final_deepswarm_model
from generic_tpot import reformat_data_traintest
from sklearn.metrics import r2_score
import scipy.stats as sp
from keras.initializers import glorot_uniform
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
import autokeras
import torch
import pickle

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# Data Process

In [2]:
rawdata = pd.read_csv('./AutoML_Data_Process/R_P_CDS.csv')
rawdata.at[0, 'Seq'] = rawdata.iloc[0].Seq*5
rawdata.Seq = rawdata.Seq.apply(lambda x:x[:826])
rawdata.to_csv('./output/experimental_data_fineturn.csv', index=False, encoding = 'utf_8_sig')
rawdata

,Gene,Seq,OD
0,atoB(thl)_100,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGCCGTCG...,11.688265
1,atoB(thl)_130,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCCATAATT...,12.008913
2,atoB(thl)_140,CGCGCCTTGACATAAAGTCTAACCTATAGGTATAATGTGTGGATCT...,9.565730
3,atoB(thl)_150,CGCGCCTTGACAATTAATCATCCGGCTCGTATAATGTGTGGAATTG...,11.556572
4,atoB(thl)_160,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGCTTAAT...,9.913603
...,...,...,...
562,GFP_320,CGCGCCTTGACATTTATCCCTTGCGGCGATATAATGTGTGGATAAG...,10.105502
563,GFP_40,CGCGCCTTGACATAAAGTCTAACCTATAGGCATAATTATTTCATCC...,9.076736
564,GFP_50,CGCGCCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCACGAA...,8.622338
565,GFP_70,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCTATAATA...,9.005457


In [10]:
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr

def calculate_metrics(preds, y):
    """
    Calculate 'R2', 'Pearson', and 'Spearman' metrics.

    Parameters:
    - preds: Predicted values
    - y: True values

    Returns:
    - r2: R-squared (R2) score
    - pearson: Pearson correlation coefficient
    - spearman: Spearman rank correlation coefficient
    """
    # R-squared (R2) score
    r2 = r2_score(y, preds)

    # Pearson correlation coefficient
    pearson, _ = pearsonr(preds, y)

    # Spearman rank correlation coefficient
    spearman, _ = spearmanr(preds, y)

    return r2, pearson, spearman


# Example 1: Transfer Learning on a DeepSwarm Model 

In [11]:
# Load DeepSwarm Model and freeze all except last two layers (randomly chose this - feel free to customize)
final_model_path = './BioAutoMATED/literature/outputs/deepswarm/regression/'
final_model_name = 'deepswarm_deploy_model.h5'
# get sequences with help from https://stackoverflow.com/questions/53183865/unknown-initializer-glorotuniform-when-loading-keras-model
with CustomObjectScope({'GlorotUniform': glorot_uniform(), 'BatchNormalizationV1': BatchNormalization()}): # , 'BatchNormalizationV1': BatchNormalization()
    model = tf.keras.models.load_model(final_model_path + final_model_name)
print(model.summary())
print('model is originally trainable: ' + str(model.trainable))
print('number of layers in the model: ' + str(len(model.layers)))

# set all layers except last two dense ones to be fixed
for layer_idx, layer in enumerate(model.layers):
    if layer_idx > len(model.layers) - 3:
        print(str(layer_idx) + ': ' + str(layer) + ', keeping trainable = ' + str(layer.trainable))
    else:
        layer.trainable = False
        print(str(layer_idx) + ': ' + str(layer) + ', setting trainable to ' + str(layer.trainable))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1692707059.049456 (InputLaye (None, 826, 4, 1)         0         
_________________________________________________________________
1692707059.0506542 (Conv2D)  (None, 826, 4, 64)        640       
_________________________________________________________________
1692707059.068253 (Conv2D)   (None, 826, 4, 8)         12808     
_________________________________________________________________
1692707059.082017 (Flatten)  (None, 26432)             0         
_________________________________________________________________
1692707059.0864587 (Dense)   (None, 1)                 26433     
Total params: 39,881
Trainable params: 39,881
Non-trainable params: 0
_________________________________________________________________
None
model is originally trainable: True
number of layers in the model: 5
0: <tensorflow.python.keras.engine.input_layer.InputLayer object

#### 微调前

In [12]:
# Transform the test set RBS data to fine-tune this model
data_folder = './output/'
data_file = 'experimental_data_fineturn.csv'

# Give inputs for data generation
input_col = 'Seq'
target_col = 'OD'
pad_seqs = 'max'
augment_data = 'none'
sequence_type = 'nucleic_acid'
task = 'regression'
model_type = 'deepswarm'

# allows user to interpret model with data not in the original training set
# so apply typical cleaning pipeline
df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)
    
# format data inputs appropriately for autoML platform    
numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)

# transform output (target) into bins for classification
transformed_output, transform_obj = transform_regression_target(df_data_output)
    
# now, we have completed the pre-processing needed to feed our data into deepswarm
# deepswarm input: numerical_data_input
# deepswarm output: transformed_output
X = numerical_data_input
y = transformed_output

Confirmed: All sequence characters are in alphabet
Padding all sequences to a length of 826
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.


In [13]:
# 使用微调前的模型进行预测
preds = model.predict(X)
r2, pearson, spearman = calculate_metrics(preds, y)
print(f"R2: {r2}")
print(f"Pearson: {pearson}")
print(f"Spearman: {spearman}")

R2: -1.319840171329898
Pearson: [0.01151065]
Spearman: 0.002300183888643859


#### 微调后

In [14]:
# Transform the test set RBS data to fine-tune this model
data_folder = './output/'
data_file = 'experimental_data_fineturn.csv'

# Give inputs for data generation
input_col = 'Seq'
target_col = 'OD'
pad_seqs = 'max'
augment_data = 'none'
sequence_type = 'nucleic_acid'
task = 'regression'
model_type = 'deepswarm'

# allows user to interpret model with data not in the original training set
# so apply typical cleaning pipeline
df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)
    
# format data inputs appropriately for autoML platform    
numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)

# transform output (target) into bins for classification
transformed_output, transform_obj = transform_regression_target(df_data_output)
    
# now, we have completed the pre-processing needed to feed our data into deepswarm
# deepswarm input: numerical_data_input
# deepswarm output: transformed_output
X = numerical_data_input
y = transformed_output

Confirmed: All sequence characters are in alphabet
Padding all sequences to a length of 826
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.


In [15]:
# 数据集大小
total_samples = X.shape[0]
train_samples = int(0.8 * total_samples)
test_samples = total_samples - train_samples

# 随机打乱数据索引
indices = np.arange(total_samples)
np.random.shuffle(indices)

# 根据8:2划分比例获取训练集和测试集的索引
train_indices = indices[:train_samples]
test_indices = indices[train_samples:]

# 根据索引划分数据集
X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]

# 打印数据集形状
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (453, 826, 4, 1)
y_train shape: (453, 1)
X_test shape: (114, 826, 4, 1)
y_test shape: (114, 1)


In [16]:
finetune_model_name = 'fine_tune_deepswarm_deploy_model.h5'
    
print('Fitting final model now...')
num_epochs = 30 # can choose how many epochs you want
deploy_model = fit_final_deepswarm_model(model, task, num_epochs,  X_train, y_train)
        
# Save the final deploy trained model
deploy_model.save(final_model_path + finetune_model_name)
print_summary(deploy_model, final_model_path + 'fine_tune_model_topology.txt')
print(deploy_model.summary())

Fitting final model now...
Train on 407 samples, validate on 46 samples
Epoch 1/30
407/407 [==============================] - 1s 3ms/sample - loss: 0.4568 - mean_squared_error: 0.4568 - val_loss: 0.2800 - val_mean_squared_error: 0.2800
Epoch 2/30
407/407 [==============================] - 1s 3ms/sample - loss: 0.3285 - mean_squared_error: 0.3285 - val_loss: 0.2813 - val_mean_squared_error: 0.2813
Epoch 3/30
407/407 [==============================] - 1s 3ms/sample - loss: 0.2836 - mean_squared_error: 0.2836 - val_loss: 0.2447 - val_mean_squared_error: 0.2447
Epoch 4/30
407/407 [==============================] - 1s 3ms/sample - loss: 0.2416 - mean_squared_error: 0.2416 - val_loss: 0.2124 - val_mean_squared_error: 0.2124
Epoch 5/30
407/407 [==============================] - 1s 3ms/sample - loss: 0.2066 - mean_squared_error: 0.2066 - val_loss: 0.2010 - val_mean_squared_error: 0.2010
Epoch 6/30
407/407 [==============================] - 1s 3ms/sample - loss: 0.1841 - mean_squared_error: 0.1

In [21]:
# 使用微调前的模型进行预测
preds = deploy_model.predict(X_test)
r2, pearson, spearman = calculate_metrics(preds, y_test)
print(f"R2: {r2}")
print(f"Pearson: {pearson}")
print(f"Spearman: {spearman}")

R2: 0.4511488891585621
Pearson: [0.69088473]
Spearman: 0.7262023855328974


# Example 2: Transfer Learning on an AutoKeras Model 

In [23]:
data_folder = './output/'
data_file = 'experimental_data_fineturn.csv'


# Give inputs for data generation
input_col = 'Seq'
target_col = 'OD'
pad_seqs = 'max'
augment_data = 'none'
sequence_type = 'nucleic_acid'
task = 'regression'
model_type = 'autokeras'

# allows user to interpret model with data not in the original training set
# so apply typical cleaning pipeline
df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)
    
# format data inputs appropriately for autoML platform    
numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)

# Format data inputs appropriately for autoML platform
transformed_output, transform_obj = transform_regression_target(df_data_output)

# now, we have completed the pre-processing needed to feed our data into autokeras
# autokeras input: oh_data_input
# autokeras output: transformed_output
X = oh_data_input
y = transformed_output # don't convert to categorical for autokeras

Confirmed: All sequence characters are in alphabet
Padding all sequences to a length of 826
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.


In [24]:
final_model_path = './BioAutoMATED/literature/models/autokeras/regression/'
final_model_name = 'optimized_autokeras_pipeline_regression.h5'

In [29]:
train_size = 0.85
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, np.array(y).astype(float),train_size=train_size, test_size = 1-train_size)

clf = autokeras.utils.pickle_from_file(final_model_path+final_model_name)

evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))

preds = clf.predict(np.array(X_test_new))
r2 = r2_score(np.array(y_test_new), preds)
print("R-squared after no retraining: ", r2)
print('Evaluation after no retraining: ', evaluation)
r2, pearson, spearman = calculate_metrics(preds, np.array(y_test_new).flatten())
print(f"R2: {r2}")
print(f"Pearson: {pearson}")
print(f"Spearman: {spearman}")


# retrain = False indicates that the weights should be reused and then retrained
# retrain = True indicates that the weights should be reinitialized from scratch
# this may seem unintuitive but in the documentation, retrain is a boolean indicating whether or not to reinitialize the weights of the model
clf.fit(np.array(X_train_new),np.array(y_train_new), retrain=False)
evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))
preds = clf.predict(np.array(X_test_new))
r2 = r2_score(np.array(y_test_new), preds)
print("R-squared after some retraining: ", r2)
print('Evaluation after some retraining: ', evaluation)
r2, pearson, spearman = calculate_metrics(preds, np.array(y_test_new).flatten())
print(f"R2: {r2}")
print(f"Pearson: {pearson}")
print(f"Spearman: {spearman}")


# can save and reload at will
autokeras.utils.pickle_to_file(clf, final_model_path + 'fine_tune_autokeras_pipeline_classification.h5')
test = autokeras.utils.pickle_from_file(final_model_path+'fine_tune_autokeras_pipeline_classification.h5')

# showing retrain = True wipes the old weights and ends up with a worse model
clf.fit(np.array(X_train_new),np.array(y_train_new), retrain=True)
evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))
preds = clf.predict(np.array(X_test_new))
r2 = r2_score(np.array(y_test_new), preds)
print("R-squared after training weights from scratch: ", r2)
print('Evaluation after training weights from scratch: ', evaluation)

r2, pearson, spearman = calculate_metrics(preds, np.array(y_test_new).flatten())
print(f"R2: {r2}")
print(f"Pearson: {pearson}")
print(f"Spearman: {spearman}")

R-squared after no retraining:  -0.08753596947344922
Evaluation after no retraining:  0.3797411622989434
R2: -0.08753596947344922
Pearson: 0.1878518761272091
Spearman: 0.15101655738478229
R-squared after some retraining:  0.20879726410383748
Evaluation after some retraining:  0.27626879016128747
R2: 0.20879726410383748
Pearson: 0.5160097091631651
Spearman: 0.5120713241190622
R-squared after training weights from scratch:  0.3874438134200159
Evaluation after training weights from scratch:  0.21388975150671605
R2: 0.3874438134200159
Pearson: 0.6349562184367432
Spearman: 0.6535874333694985


# Part 3: Transfer Learning on TPOT Model

In [34]:
# read in data file
data_folder = './output/'
data_file = 'experimental_data_fineturn.csv'

# give inputs for data generation
input_col_name = 'Seq'
target_col = 'OD'
pad_seqs = 'max'
augment_data = 'none'
sequence_type = 'nucleic_acid'
task = 'regression'
model_type = 'tpot'

# allows user to interpret model with data not in the original training set
# so apply typical cleaning pipeline
df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)
    
# format data inputs appropriately for autoML platform    
numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)

# Format data inputs appropriately for autoML platform
transformed_output, transform_obj = transform_regression_target(df_data_output)

X = numerical_data_input
y = transformed_output # don't convert to categorical for tpot
training_features, training_target = reformat_data_traintest(X, y)
train_size = 0.85
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(training_features, training_target, train_size=train_size, test_size = 1-train_size)

Confirmed: All sequence characters are in alphabet
Padding all sequences to a length of 826
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.


In [35]:
# give inputs for paths
final_model_path = './BioAutoMATED/literature/outputs/tpot/regression/'
final_model_name = 'final_model_tpot_regression.pkl'
output_folder = final_model_path

with open(final_model_path+final_model_name, 'rb') as file:  
    model = pickle.load(file)

In [36]:
# partial_fit transfer learning is only possible for models that support it - most do not
# see reference list of those models here: https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning
try:
    model.partial_fit(X_train_new,y_train_new)
except:
    print("No partial_fit could be applied. Trying warm_start instead.")
    print("")
try:
    preds = model.predict(X_test_new)
    
    r2 = r2_score(np.array(y_test_new), preds)
    print('Original model on new test data R2 : ', r2)
    print('Original model on new test data: ', sp.pearsonr(y_test_new, preds))
    
    
    r2, pearson, spearman = calculate_metrics(preds, np.array(y_test_new).flatten())
    print(f"R2: {r2}")
    print(f"Pearson: {pearson}")
    print(f"Spearman: {spearman}")
    
    print('Keys that must be manually changed in the model to allow fine-tuning on new data: ')
    for key in list(model.get_params().keys()):
        if 'warm_start' in key or 'n_estimator' in key:
            print('\t' + key)
        model.set_params(warm_start = True)
        model.set_params(n_estimators = 1 + model.get_params()['n_estimators'])

    model.fit(X_train_new,y_train_new)
    preds = model.predict(X_test_new)
    r2 = r2_score(np.array(y_test_new), preds)
    print('Fine-tuned model on new test data R2 : ', r2)    
    print('Fine-tuned model on new test data: ', sp.pearsonr(y_test_new, preds))
    r2, pearson, spearman = calculate_metrics(preds, np.array(y_test_new).flatten())
    print(f"R2: {r2}")
    print(f"Pearson: {pearson}")
    print(f"Spearman: {spearman}")
except Exception as e:
    print(e)
    print("No warm_start could be applied. Model is not compatible with transfer learning.")


No partial_fit could be applied. Trying warm_start instead.

Original model on new test data R2 :  -0.32680605057210044
Original model on new test data:  (0.25109081649536685, 0.019703911226954935)
R2: -0.32680605057210044
Pearson: 0.25109081649536685
Spearman: 0.2594367658851832
Keys that must be manually changed in the model to allow fine-tuning on new data: 
	n_estimators
	warm_start
Fine-tuned model on new test data R2 :  0.28544609526852893
Fine-tuned model on new test data:  (0.5834482839037382, 3.738224773868991e-09)
R2: 0.28544609526852893
Pearson: 0.5834482839037382
Spearman: 0.605207792820416


# Part 4: Transfer Learning on AutoKeras Toehold Regression Model + Testing

In [ ]:
task = 'regression'
pad_seqs = 'max'
augment_data = 'none'
sequence_type = 'nucleic_acid'
model_type = 'autokeras'
final_model_path = './BioAutoMATED/literature/models/autokeras/regression/'

## With retrained Green et al. models, predict on test sets (Pardee et al.)

In [ ]:
pearsons = []
spearmans = []
R2s = []
for i in range(25): # do any # of trials
    print("--------------  ROUND " + str(i) + " --------------")
    # Read in Green et al. data file
    data_folder = './output/'
    data_file = 'experimental_data_fineturn.csv'

    # Give inputs for data generation
    input_col = 'seq'
    target_col = 'target'
    final_model_name = 'optimized_autokeras_pipeline_regression.h5'

    # allows user to interpret model with data not in the original training set
    # so apply typical cleaning pipeline
    df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)

    # Format data inputs appropriately for autoML platform
    numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)
    transformed_output, transform_obj = transform_regression_target(df_data_output)

    # now, we have completed the pre-processing needed to feed our data into autokeras
    # autokeras input: oh_data_input
    # autokeras output: transformed_output
    X = oh_data_input
    y = transformed_output # don't convert to categorical for autokeras

    train_size = 0.9
    X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, np.array(y).astype(float),train_size=train_size, test_size = 1-train_size)

    clf = autokeras.utils.pickle_from_file(final_model_path+final_model_name)
    evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))

    # retrain = False indicates that the weights should be reused and then retrained
    # retrain = True indicates that the weights should be reinitialized from scratch
    clf.fit(np.array(X_train_new),np.array(y_train_new), retrain=False)
    evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))
    y_pred = clf.predict(np.array(X_test_new))
    r2 = r2_score(np.array(y_test_new), y_pred)
    R2s.append(r2)


--------------  ROUND 0 --------------
Confirmed: All sequence characters are in alphabet
Padding all sequences to a length of 826
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.


In [ ]:
print(np.mean(pearsons))
print(np.std(pearsons))
print(np.mean(spearmans))
print(np.std(spearmans))
print(np.mean(R2s))
print(np.std(R2s))

0.6487315269228773
0.016604972636010205
0.6456944583216552
0.016758170318431506
-47.93336678706019
0.28280402583622866
